### Importing libraries

In [1]:
from ImgOps import *
import numpy as np
import cv2 as cv
from sklearn.cluster import DBSCAN

### Imreading images

In [2]:
query_img = cv.imread('query_4.jpg')
query_img_gray = cv.cvtColor(query_img, cv.COLOR_BGR2GRAY)
imshow(query_img)

train_img = cv.imread('train_4.jpg')
train_img_gray = cv.cvtColor(train_img, cv.COLOR_BGR2GRAY)
imshow(train_img)

### Adding keypoints via SIFT

In [3]:
# Initiate SIFT detector
sift = cv.SIFT_create()

# find the keypoints and descriptors with SIFT
query_kps, query_des = sift.detectAndCompute(query_img_gray, None)
train_kps, train_des = sift.detectAndCompute(train_img_gray, None)

### Keypoints matching via FLANN

In [4]:
index_params = dict(algorithm=1, trees=5)
search_params = dict(checks=50)
flann = cv.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(query_des, train_des, k=2)
matchesMask = [[0, 0] for i in range(len(matches))]

good_matches = []
for i, (m, n) in enumerate(matches):
    if m.distance < .5 * n.distance:
        matchesMask[i] = [1, 0]
        good_matches.append(m)
good_matches = np.asarray(good_matches)

draw_params = dict(matchColor=(0, 255, 0),
                   matchesMask=matchesMask,
                   flags=cv.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
matched_img = cv.drawMatchesKnn(query_img, query_kps, train_img, train_kps, matches, None, **draw_params)

imshow(matched_img)

### Clustering via DBSCAN

In [6]:
x_pts = np.float32([query_kps[des.queryIdx].pt[0] for des in good_matches])
y_pts = np.float32([query_kps[des.queryIdx].pt[1] for des in good_matches])

good_crd = []
for x, y in zip(x_pts, y_pts):
    good_crd.append([x, y])

cluster_gps = DBSCAN(eps=50, min_samples=5).fit_predict(good_crd)

group_pts = {}
for gp, pt in zip(cluster_gps, good_crd):
    if gp == -1: continue
    else:
        if gp not in group_pts:
            group_pts[gp] = [pt]
        else:
            group_pts[gp].append(pt)

{0: [[286.53848, 262.5918],
  [286.53848, 262.5918],
  [288.9568, 307.51038],
  [288.9568, 307.51038],
  [289.6289, 282.88913],
  [289.6289, 282.88913],
  [292.17508, 262.6499],
  [292.17508, 262.6499],
  [294.75516, 294.56625],
  [296.60086, 287.8304],
  [298.2112, 269.9421],
  [298.35812, 262.30304],
  [299.84317, 294.48398],
  [299.84317, 294.48398],
  [299.84317, 294.48398],
  [303.2797, 266.1803],
  [303.2797, 266.1803],
  [308.72174, 294.29886],
  [310.05545, 266.37903],
  [310.05545, 266.37903],
  [320.2638, 275.28452],
  [320.2638, 275.28452],
  [339.07874, 264.37003]],
 1: [[575.65515, 140.9867],
  [580.5156, 190.17245],
  [580.5156, 190.17245],
  [586.9829, 172.07022],
  [596.8636, 163.81593],
  [598.7796, 134.2571],
  [605.5413, 103.86533]],
 2: [[655.5125, 64.98112],
  [655.6221, 57.968853],
  [661.8777, 81.8682],
  [662.9233, 53.46857],
  [663.77374, 181.06851],
  [665.76385, 83.77432],
  [665.76385, 83.77432],
  [666.9806, 59.17556],
  [667.18066, 65.44178],
  [667.18066,

### Show clusterized image

In [6]:
for pts in group_pts.values():
    for x, y in pts:
        clust_img = cv.circle(query_img, (int(x), int(y)), radius=3, color=(255, 0, 255), thickness=-1)
    imshow(clust_img)
    clust_img = None

### Feature Homography

In [22]:
dst = np.float32([train_kps[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

for key in group_pts:
    src = np.asarray(group_pts[key]).reshape(-1, 1, 2)
    M, _ = cv.findHomography(dst, src, cv.RANSAC, 5.)
    h, w = train_img.shape
    pts = np.float32([[0, 0], [0, h], [w, h], [w, 0]]).reshape(-1, 1, 2)
    dst = cv.perspectiveTransform(pts, M)
    res_img = cv.polylines(query_img, [np.int32(dst)], True, 255, 3, cv.LINE_AA)

[[[286.53848 262.5918 ]]

 [[286.53848 262.5918 ]]

 [[288.9568  307.51038]]

 [[288.9568  307.51038]]

 [[289.6289  282.88913]]

 [[289.6289  282.88913]]

 [[292.17508 262.6499 ]]

 [[292.17508 262.6499 ]]

 [[294.75516 294.56625]]

 [[296.60086 287.8304 ]]

 [[298.2112  269.9421 ]]

 [[298.35812 262.30304]]

 [[299.84317 294.48398]]

 [[299.84317 294.48398]]

 [[299.84317 294.48398]]

 [[303.2797  266.1803 ]]

 [[303.2797  266.1803 ]]

 [[308.72174 294.29886]]

 [[310.05545 266.37903]]

 [[310.05545 266.37903]]

 [[320.2638  275.28452]]

 [[320.2638  275.28452]]

 [[339.07874 264.37003]]]


error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-9d_dfo3_\opencv\modules\calib3d\src\fundam.cpp:380: error: (-215:Assertion failed) src.checkVector(2) == dst.checkVector(2) in function 'cv::findHomography'


In [20]:
src.shape

(23, 1, 2)

In [21]:
dst.shape

(154, 1, 2)